In [ ]:
import requests
import base64
from PIL import Image
import io
import cv2
import numpy as np
import glob
import os
import imutils

def encode(val):
    val=val.replace("+","BGABG")
    val=val.replace("=","AGAAGA")
    val=val.replace("/","OPAWQQQQ")
    return val

def decode(val):
    val=val.replace("BGABG","+")
    val=val.replace("AGAAGA","=")
    val=val.replace("OPAWQQQQ","/")
    return val
def stringToImage(base64_string):
    imgdata = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(imgdata))
# convert PIL Image to an RGB image( technically a numpy array ) that's compatible with opencv
def toRGB(image):
    return cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)


imgs=glob.glob('imgs//*.jpg')
for i in imgs:
    img=cv2.imread(i)
    h,w,c=img.shape
    #if w>500:
    #    img=imutils.resize(img,width=500)
    _,buffer=cv2.imencode('.jpg',img)
    img_encoded=base64.b64encode(buffer)
    sttt=(str(img_encoded)[2:])[:-1]
    e=encode(sttt)
    res=requests.get(f"http://127.0.0.1:5000/knee?image={e}")

    img_trans=decode(res.json()["img_st"])

    img_transparent = toRGB(stringToImage(img_trans))
    cv2.imwrite('img_trans.jpg',img_transparent)
    cv2.imshow('img',cv2.add(img,img_transparent))
    cv2.waitKey(0)
    cv2.destroyAllWindows()